<a href="https://colab.research.google.com/github/MogarampalliHema/sithafal/blob/main/chat_to_pdftask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### Loading Documents

In [ ]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data


In [ ]:
# wikipedia
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data


### Chunking Data

In [ ]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks


### Calculating Cost

In [ ]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

### Embedding and Uploading to a Vector Database (Pinecone)

In [ ]:
def insert_or_fetch_embeddings(index_name, chunks):
    # importing the necessary libraries and initializing the Pinecone client
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec


    pc = pinecone.Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))


    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536, api_key=os.environ.get("OPEN_AI_KEY"))  # 512 works as well

    # loading from existing index
    if index_name in pc.list_indexes().names():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        # creating the index and embedding the chunks into the index
        print(f'Creating index {index_name} and embeddings ...', end='')

        # creating a new index
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
        )
        )

        # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # inserting the embeddings into the index and returning a new Pinecone vector store object.
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')

    return vector_store


In [ ]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')


### Asking and Getting Answers

In [ ]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1, api_key=os.environ.get("OPEN_AI_KEY"))

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.invoke(q)
    return answer


### Running Code

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

#### Ask a PDF

In [ ]:
data = load_document('./us_constitution.pdf')
# print(data[1].page_content)
# print(data[10].metadata)

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[20].page_content)} characters in the page')

Loading ./us_constitution.pdf
You have 41 pages in your data
There are 1174 characters in the page


In [ ]:
# data = load_document('files/the_great_gatsby.docx')
# print(data[0].page_content)

In [ ]:
# data = load_from_wikipedia('GPT-4', 'de')
# print(data[0].page_content)

In [ ]:
chunks = chunk_data(data)
print(len(chunks))
# print(chunks[10].page_content)

224


In [ ]:
print_embedding_cost(chunks)

Total Tokens: 9842
Embedding Cost in USD: 0.000197


In [ ]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [ ]:
index_name = 'askadocument'
# vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Index askadocument already exists. Loading embeddings ... Ok


#### While Loop for Asking Questions

In [ ]:
import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = ask_and_get_answer(vector_store, q)
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')



In [ ]:
# Install required libraries
!apt-get install -y tesseract-ocr
!pip install pdfplumber pytesseract pillow pandas


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Imports
import pdfplumber
import pytesseract
from PIL import Image, ImageEnhance, ImageOps
import pandas as pd
import re

# Function to clean and normalize text
def clean_text(text):
    """
    Clean extracted text by removing excessive spaces, newlines, and invalid symbols.
    """
    text = re.sub(r'\n+', ' ', text)  # Replace newlines
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII
    text = re.sub(r'[^\w\s\.\%\:\$\-]', '', text)  # Keep valid symbols
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    return text

# Function to preprocess images for OCR
def preprocess_image_for_ocr(image):
    """
    Preprocess image: grayscale, threshold, contrast enhancement, and resizing.
    """
    image = image.convert("L")  # Grayscale
    image = ImageEnhance.Contrast(image).enhance(2.5)  # Increase contrast
    image = ImageOps.invert(image)  # Invert for better text visibility
    image = image.resize((image.width * 2, image.height * 2), Image.Resampling.LANCZOS)  # Resize
    return image

# OCR function with fallback to different PSM modes
def ocr_page(pdf_path, page_number):
    """
    Perform OCR with preprocessing and adaptive PSM modes.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[page_number]
            image = page.to_image(resolution=300).original
            image = preprocess_image_for_ocr(image)

            # First OCR attempt with PSM 6
            text = pytesseract.image_to_string(image, config="--psm 6")
            if not text.strip():
                # Fallback to PSM 4 for better layout parsing
                text = pytesseract.image_to_string(image, config="--psm 4")
            print(f"OCR performed on Page {page_number + 1}")
            return clean_text(text)
    except Exception as e:
        return f"Error performing OCR on Page {page_number + 1}: {e}"

# Function to dynamically format and align OCR output
def format_ocr_output(text):
    """
    Dynamically format OCR output into structured lines with labels and values.
    """
    # Patterns for degrees, percentages, and monetary values
    degree_pattern = r'(Doctoral|Professional|Masters|Bachelors|Associates|Some college|High school|Less than high school)\s(degree|diploma)'
    value_pattern = r'(\d+\.\d+\%|\$\d+|\d+)'

    # Find all degrees and values dynamically
    degrees = re.findall(degree_pattern, text)
    values = re.findall(value_pattern, text)

    formatted_output = []
    idx = 0

    # Align degrees with their respective values
    for degree in degrees:
        degree_text = " ".join(degree)
        value = values[idx] if idx < len(values) else "N/A"
        formatted_output.append(f"{degree_text}: {value}")
        idx += 1

    # Print formatted lines
    for line in formatted_output:
        print(line)

# Function to extract text from specific pages
def extract_page_text(pdf_path, page_numbers):
    """
    Extract text from specific pages using OCR as a fallback.
    """
    page_data = {}
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in page_numbers:
            try:
                page = pdf.pages[page_num]
                text = page.extract_text()
                if text and len(text.strip()) > 0:
                    page_data[f"Page {page_num+1}"] = clean_text(text)
                else:
                    print(f"No text found on Page {page_num+1}, performing OCR...")
                    page_data[f"Page {page_num+1}"] = ocr_page(pdf_path, page_num)
            except IndexError:
                page_data[f"Page {page_num+1}"] = "Page number out of range."
    return page_data

# Function to extract tabular data
def extract_table_data(pdf_path, page_number):
    """
    Extract and clean table data from a specific page.
    """
    try:
        with pdfplumber.open(pdf_path) as pdf:
            page = pdf.pages[page_number]
            tables = page.extract_tables()
            if tables:
                cleaned_table = [[re.sub(r'\s+', ' ', str(cell).strip()) for cell in row] for row in tables[0]]
                df = pd.DataFrame(cleaned_table[1:], columns=cleaned_table[0])
                print(f"Table extracted successfully from Page {page_number + 1}")
                return df
            else:
                print(f"No tables found on Page {page_number + 1}")
                return pd.DataFrame()
    except Exception as e:
        print(f"Error extracting table from Page {page_number + 1}: {e}")
        return pd.DataFrame()

# PDF File Path
pdf_path = "sample.pdf"

# Extract text and format Page 2
print("----- Page 2: Unemployment Information by Degree -----")
specific_pages = extract_page_text(pdf_path, [1, 5])
page_2_text = specific_pages.get("Page 2", "No data found.")
format_ocr_output(page_2_text)

# Extract and display tabular data from Page 6
print("\n----- Page 6: Tabular Data -----")
page_6_table = extract_table_data(pdf_path, 5)
if not page_6_table.empty:
    print("Extracted Tabular Data:")
    print(page_6_table)
else:
    print("No tabular data found on Page 6.")


----- Page 2: Unemployment Information by Degree -----
No text found on Page 2, performing OCR...
OCR performed on Page 2
Professional degree: 2013
Masters degree: 2013
Bachelors degree: 2
Associates degree: 4
High school diploma: 5

----- Page 6: Tabular Data -----
Table extracted successfully from Page 6
Extracted Tabular Data:
                                                Year      2010      2011  \
0                                     All Industries  26093515  27535971   
1                                      Manufacturing   4992521   5581942   
2   Finance, Insurance, Real Estate, Rental, Leasing   4522451   4618678   
3  Arts, Entertainment, Recreation, Accommodation...    964032   1015238   
4                                              Other  15614511  16320113   

       2012      2013      2014      2015  
0  28663246  29601191  30895407  31397023  
1   5841608   5953299   6047477   5829554  
2   4797313   5031881   5339678   5597018  
3   1076249   1120496   1189646   1